<div id='id0' > </div>

<body>
    <h1> <b> Análisis Exploratorio de Datos </b> </h1> 
    <h2> <b> Índice </b> </h2>
</body>

1.  [Generalidades](#id1)
2.  [Características Preliminares de las Fuentes de Datos](#id2)
3.  [Imoportación de la Librerías Requeridas](#id3)
4.  [Extración de los Datos](#id4)
5.  [Carga de los Datos en Dataframes](#id5)
6.  [Fuente de Datos: Títulos de Amazon Prime](#id6)
7.  [Fuente de Datos: Títulos de Disney Plus](#id7)
8.  [Fuente de Datos: Salas de Cines](#id8)
9.  [Unión de los Dataframes](#id9)
10. [Obtención de los Insights](#id10)

---


<div id='id1' > </div>

<body>
     <h1> <b> Generalidades </b> <br> </h1>
     <p> Los datos fueron provistos en archivos de diferentes extensiones, como csv o json. La información ha de ser procesada y normalizada para cada dataset, entre otras tareas, para obtener información relevante, tal como: <p>
     <ul>
          <li> <p> 
          Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])</b>. </li>
          <li> <p> 
          Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)</b>. </li>
          <li> <p> 
          Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')</b>. </li>
          <li> <p> 
          Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)</b>. </li>
     </ul> 
</body>




[ Índice </p> ](#id0)

---

<div id='id2' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Características Preliminares de las Fuentes de Datos </b> <br> </h1>
     <p>
     Los datasets disponibles corresponden ha los titulos disponibles en cuatro proveedores de streaming. <br>
     <br>
     Los recursos recuperados de las fuentes fueron:
          <ul>
               <li> <p> 
               Títulos de Amazon Prime: amazon_prime_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Disney Plus: disney_plus_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Hulu: hulu_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Netflix: netflix_titles.json</b>. </li>
               </ul> 
     </p>
</body>

[ Índice </p> ](#id0)

---

<div id='id3' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Imoportación de la Librerías Requeridas </b> <br> </h1>
</body>

In [1]:
#Importar las librerías
#Python 
import os
from pathlib import Path
from datetime import datetime

#Request
import requests

#Pandas
import pandas as pd

[ Índice </p> ](#id0)

---

<div id='id4' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Extración de los Datos </b> <br> </h1>
</body>

In [2]:
#Definir los directorios y las rutas
MAIN_DIR = (Path(os.path.abspath(''))).parent                                    #out: "$PWD/streaming_catalog_api"
DATA_DIR = Path(f"{MAIN_DIR}/data")                                              #out: "$PWD/streaming_catalog_api/data"
EXT_DIR =  Path(f"{DATA_DIR}/external")                                          #out: "$PWD/streaming_catalog_api/data/external"
INT_DIR = Path(f"{DATA_DIR}/interim")                                            #out: "$PWD/streaming_catalog_api/data/interim"
PRO_DIR = Path(f"{MAIN_DIR}/processed")                                          #out: "$PWD/streaming_catalog_api/data/raw"

In [ ]:
print(
    "MAIN_DIR:" f"{MAIN_DIR}\n",
    "DATA_DIR:" f"{DATA_DIR}\n",
    "EXT_DIR:" f"{EXT_DIR}\n",
    "INT_DIR:" f"{INT_DIR}\n",
    "PRO_DIR:" f"{PRO_DIR}\n"
)

In [3]:
#Crear una lista de diccionarios para almacenar la categoría y la url
category_list = [
    {
        "name": "amazon_prime_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/amazon_prime_titles.csv"
    },
    {
        "name": "disney_plus_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/disney_plus_titles.csv"
    },
    {
        "name": "hulu_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/hulu_titles.csv"
    },
    {
        "name": "netflix_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/netflix_titles.json"
    }
    ]

In [4]:
#Definir la función get_data: esta función obtiene mediante web scraping las fuentes de datos
def get_data(category_list):
    for i in range(len(category_list)):
        streaming_plataform = category_list[i]["name"]
        if streaming_plataform != "netflix_titles":
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"
        else:
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.json"
        
        now = datetime.now()
        category = category_list[i]["name"]
        year = now.year
        month = now.month
        day = now.day
        
        r = requests.get(category_list[i]["url"])
        r.encoding = 'utf-8'
        FILE_NAME = file_name.format(
            category = category, 
            year = year, 
            month = month, 
            day = day
            )

        FILE_PATH = EXT_DIR/FILE_NAME
        FILE_PATH.parent.mkdir(
            parents=True,
            exist_ok=True
            )
            
        with open(FILE_PATH,'w') as file_outp:
            file_outp.write(r.text)
            print(f"Se ha guardado el archivo en el directorio: {FILE_PATH} \n")

In [5]:
#Ejecutar la función get data
get_data(category_list = category_list)

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\amazon_prime_titles\2022-12\amazon_prime_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\disney_plus_titles\2022-12\disney_plus_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\hulu_titles\2022-12\hulu_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\netflix_titles\2022-12\netflix_titles-08-12-2022.json 



[ Índice </p> ](#id0)

---

<div id='id5' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Carga de los Datos en Dataframes </b> <br> </h1>
</body>

<div id='id6' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h2> <b> Fuente de Datos</b>: Títulos de amazon prime </h2>
</body>

In [6]:
#Declarar los parámetros para la carga del dataframe
path = "c:/Users/Jose Salas/OneDrive/Escritorio/streaming_catalog_api/data/external/amazon_prime_titles/2022-12/amazon_prime_titles-08-12-2022.csv"
sep = ","
cod = "utf-8"

amz_df = pd.read_csv(filepath_or_buffer = path, delimiter = sep, encoding = cod)

In [7]:
#Visualizar preliminarmente el dataframe
amz_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [11]:
#Inspeccionar el dataframe
amz_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
release_year,9668.0,2008.341849,18.922482,1920.0,2007.0,2016.0,2019.0,2021.0


In [16]:
# Identificar variables categóricas y numércias/ Identifie cateorical and numerical vars
num_vars = amz_df.columns[amz_df.dtypes != "object"]
cat_vars = amz_df.columns[amz_df.dtypes == "object"]

print("Variables numericas:")
for var in num_vars:
    print(f"\t{var}")
print("\n")

print("Variables categóricas:")
for var in cat_vars:
    print(f"\t{var}")

Variables numericas:
	release_year


Variables categóricas:
	show_id
	type
	title
	director
	cast
	country
	date_added
	rating
	duration
	listed_in
	description


In [8]:
# Visualizar las columnas del dataframe y sus tipos de datos
amz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [14]:
#IIdentificar registros nulos
amz_df.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [15]:
#Iddentificar registros vacíos
amz_df.isna().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [20]:
#Definir las columnas que deseamos conservar
columns = [ 
            "show_id", 
            "type", 
            "title", 
            "cast", 
            "date_added", 
            "duration", 
            "listed_in"
            ]

In [21]:
#Inspeccionar preliminarmente el dataframe reducido
amz_df[columns].head()

,show_id,type,title,cast,date_added,duration,listed_in
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent","March 30, 2021",113 min,"Comedy, Drama"
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar","March 30, 2021",110 min,"Drama, International"
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...","March 30, 2021",74 min,"Action, Drama, Suspense"
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...","March 30, 2021",69 min,Documentary
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...","March 30, 2021",45 min,"Drama, Fantasy"


<body>
     <h3> <b> Inspección de datos nulos por columna de interés </b> </h3>
</body>

In [30]:
#Datos nulos, columna "cast"
amz_df.loc[amz_df["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
20,s21,TV Show,Zoboomafoo,NaN,NaN,NaN,NaN,2001,TV-Y,1 Season,Kids,Chris and Martin Kratt bring their enthusiasm ...
25,s26,Movie,Zambezi: Force of Life,Dr. Rudolf Lammers,NaN,NaN,NaN,2005,ALL,50 min,Documentary,The Zambezi river is the fourth-longest river ...
33,s34,Movie,Young Love (at the Sun Factory),NaN,NaN,NaN,NaN,2007,NR,36 min,Music Videos and Concerts,"Young Love celebrates the release of ""Too Youn..."
34,s35,Movie,Young Love (at Bowery Ballroom),NaN,NaN,NaN,NaN,2007,NR,37 min,Music Videos and Concerts,Young Love returns to Baeble's silver screen w...
70,s71,Movie,World Inferno Friendship Society,Baeble Music,NaN,NaN,NaN,2007,NR,40 min,Music Videos and Concerts,The circus comes to town with this performance...
...,...,...,...,...,...,...,...,...,...,...,...,...
9636,s9637,Movie,Pieces of a Dream: A Story of Gambling,Michelle Wong,NaN,NaN,NaN,2003,13+,49 min,"Documentary, Special Interest","When Phillip Wong died by suicide, his family ..."
9637,s9638,TV Show,Paul Newman Trilogy,NaN,NaN,NaN,NaN,2021,13+,1 Season,TV Shows,The early days of television were filled with ...
9645,s9646,TV Show,Let's Go Cozy Coupe! - Cartoons for Kids,NaN,NaN,NaN,NaN,2021,ALL,1 Season,Kids,Cozy Coupe is an online series that helps kids...
9647,s9648,TV Show,Kids In The Hall,NaN,NaN,NaN,"October 10, 2021",1995,TV-14,5 Seasons,"Arts, Entertainment, and Culture, Comedy","""The Kids in the Hall"" is a unique and bizarre..."


In [29]:
#Datos nulos, columna "date_added"
amz_df.loc[amz_df["date_added"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
16,s17,Movie,Zoombies,Glenn Miller,"Marcus Anderson, Kaiwi Lyman, Andrew Asper",NaN,NaN,2016,13+,87 min,"Horror, Science Fiction",When a strange virus quickly spreads through a...
17,s18,TV Show,Zoo Babies,NaN,Narrator - Gillian Barlett,NaN,NaN,2008,ALL,1 Season,"Kids, Special Interest",A heart warming and inspiring series that welc...
18,s19,TV Show,Zoë Coombs Marr: Bossy Bottom,NaN,Zoë Coombs Marr,NaN,NaN,2020,18+,1 Season,"Comedy, Talk Show and Variety",Zoë Coombs Marr has been on hiatus. Sort of. F...
19,s20,Movie,Zoe,Drake Doremus,"Ewan McGregor, Léa Seydoux, Theo James, Mirand...",NaN,NaN,2018,R,104 min,Science Fiction,ZOE tells a tale of forbidden love between an ...
20,s21,TV Show,Zoboomafoo,NaN,NaN,NaN,NaN,2001,TV-Y,1 Season,Kids,Chris and Martin Kratt bring their enthusiasm ...
...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve..."
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."


[ Índice </p> ](#id0)

---

<body>
     <h3> <b> Transformación del dataframe </b> </h3>
</body>

In [ ]:
#Duplicar el dataframe original conservando únicamente las columnas de interes
amz_edit_df = amz_df[columns]

<div id='id7' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h2> <b> Fuente de Datos</b>: Títulos de Disney Plus </h2>
</body>